In [94]:
import numpy as np
import matplotlib.pyplot as plt

# load train/test data from npy
data = np.load('train.npy')
labels = np.load('train_labels.npy')

# split into train and test
trainlen = int(len(labels) * 0.7)
train_data = data[:trainlen]
train_labels = labels[:trainlen]

test_data = data[trainlen:]
test_labels = labels[trainlen:]

In [95]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [96]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y_ = tf.placeholder(tf.float32, shape=[None, 26])

In [97]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [98]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [99]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [100]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [101]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [102]:
W_fc2 = weight_variable([1024, 26])
b_fc2 = bias_variable([26])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [103]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [104]:
index = 0
def next_batch(batch_size):
    global index
    result = (train_data[index:(index + batch_size)], train_labels[index:(index + batch_size)])
    index += batch_size
    index %= len(train_data)
    return result

In [105]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(50000):
    batch = next_batch(128)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: test_data, y_: test_labels, keep_prob: 1.0}))
  saver = tf.train.Saver()
  saver.save(sess, 'my_test_model',global_step=1000)

step 0, training accuracy 0.0703125
step 100, training accuracy 0.445312
step 200, training accuracy 0.476562
step 300, training accuracy 0.617188
step 400, training accuracy 0.609375
step 500, training accuracy 0.617188
step 600, training accuracy 0.523438
step 700, training accuracy 0.484375
step 800, training accuracy 0.421875
step 900, training accuracy 0.242188
step 1000, training accuracy 0.296875
step 1100, training accuracy 0.367188
step 1200, training accuracy 0.242188
step 1300, training accuracy 0.273438
step 1400, training accuracy 0.226562
step 1500, training accuracy 0.265625
step 1600, training accuracy 0.257812
step 1700, training accuracy 0.242188
step 1800, training accuracy 0.328125
step 1900, training accuracy 0.289062
step 2000, training accuracy 0.296875
step 2100, training accuracy 0.320312
step 2200, training accuracy 0.320312
step 2300, training accuracy 0.382812
step 2400, training accuracy 0.367188
step 2500, training accuracy 0.382812
step 2600, training acc

step 21600, training accuracy 0.960938
step 21700, training accuracy 0.921875
step 21800, training accuracy 0.9375
step 21900, training accuracy 0.960938
step 22000, training accuracy 0.96875
step 22100, training accuracy 0.96875
step 22200, training accuracy 0.953125
step 22300, training accuracy 0.945312
step 22400, training accuracy 0.960938
step 22500, training accuracy 0.953125
step 22600, training accuracy 0.9375
step 22700, training accuracy 0.953125
step 22800, training accuracy 0.9375
step 22900, training accuracy 0.96875
step 23000, training accuracy 0.96875
step 23100, training accuracy 0.96875
step 23200, training accuracy 0.976562
step 23300, training accuracy 0.984375
step 23400, training accuracy 1
step 23500, training accuracy 0.953125
step 23600, training accuracy 0.960938
step 23700, training accuracy 0.953125
step 23800, training accuracy 0.96875
step 23900, training accuracy 0.96875
step 24000, training accuracy 0.984375
step 24100, training accuracy 0.921875
step 2

step 43600, training accuracy 0.992188
step 43700, training accuracy 1
step 43800, training accuracy 0.992188
step 43900, training accuracy 1
step 44000, training accuracy 0.992188
step 44100, training accuracy 0.992188
step 44200, training accuracy 0.992188
step 44300, training accuracy 1
step 44400, training accuracy 0.984375
step 44500, training accuracy 1
step 44600, training accuracy 0.992188
step 44700, training accuracy 0.992188
step 44800, training accuracy 1
step 44900, training accuracy 1
step 45000, training accuracy 1
step 45100, training accuracy 1
step 45200, training accuracy 0.96875
step 45300, training accuracy 1
step 45400, training accuracy 0.984375
step 45500, training accuracy 1
step 45600, training accuracy 0.984375
step 45700, training accuracy 1
step 45800, training accuracy 1
step 45900, training accuracy 1
step 46000, training accuracy 1
step 46100, training accuracy 1
step 46200, training accuracy 0.984375
step 46300, training accuracy 1
step 46400, training 